**Autores**
- Eduardo Venegas
- Axel Magallón
- Alex Hernández
- Diego Ortiz
- Pablo Blanco

### Actividad: Protocolos de votación

**Importar las librerías necesarias**

In [1]:
import pandas as pd
import numpy as np

**Leer el documento y guardarlo dentro de un DataFrame**

In [2]:
DATA = pd.read_excel('data.xlsx', index_col=0)
DATA.head()

,Con piña,Sin piña,Mejor tacos,Mejor hamburguesas,Mejor pasta
Voter,,,,,
1,1,2,3,4,5
2,2,3,5,1,4
3,5,1,2,3,4
4,2,1,3,4,5
5,4,5,1,2,3


In [3]:
print(f"Dimensiones del DataFrame: {DATA.shape}")

Dimensiones del DataFrame: (24, 5)


**Función de votación de pluralidad**

In [4]:
def pluralidad(data: pd.DataFrame, porcentaje: bool = False) -> str and float:
    """Votación de pluralidad es la votación simple en donde gana el que más aparece en primer lugar

    Args:
        data (DataFrame): Los datos de donde se realizó la votación

    Returns:
        str: El string del ganador de la votación
        float: El porcentaje con el que ganó
    """
    winner = ''
    winner_counts = 0
    for col in data.columns:
        n = np.sum(data[col] == 1)
        if n > winner_counts:
            winner = col
            winner_counts = n
            
    if not porcentaje:
        return winner
    else:
        return winner, round(winner_counts / len(data), 2)

In [5]:
def borda(data: pd.DataFrame, ganador: bool = True) -> str:
    """Votación de borda es aquel donde se le asigna un número a cada opción, y se suman los números de las opciones que ganó cada candidato

    Args:
        data (DataFrame): Los datos de donde se realizó la votación

    Returns:
        str: El string del ganador de la votación
    """
    temp = {col: 0 for col in data.columns}
    for col in data.columns:
        for elem in data[col]:
            temp[col] += 6 - elem
            
    for category, value in temp.items():
        if value == max(temp.values()) and ganador:
            return category
        elif value == min(temp.values()) and not ganador:
            return category

In [6]:
def segunda_vuelta(data: pd.DataFrame) -> str:
    """Votación de segunda vuelta es donde se comprueba si un candidato tiene más de 50% desde un principio, en caso
        contrario se elimina el candidato con menor votación (con método de borda) y se vuelve a ejecutar el loop

    Args:
        data (DataFrame): Los datos de donde se realizó la votación

    Returns:
        str: El string del ganador de la votación
    """
    temp = data.copy()
    winner, winner_percentage = pluralidad(temp, True)
    while winner_percentage < 0.5:
        loser = borda(temp, False)
        for i, row in temp.iterrows():
            loser_value = row[loser]
            temp_row = row.copy()
            for j, temp_value in enumerate(temp_row):
                if temp_value > loser_value:
                    temp_row[j] = temp_value - 1
            temp.loc[i] = temp_row
        
        temp = temp.drop(loser, axis=1)
        winner, winner_percentage = pluralidad(temp, True)
        
    return winner

In [7]:
print(f'Votación de pluralidad: "{pluralidad(DATA)}"')
print(f'Votación de borda: "{borda(DATA)}"')
print(f'Votación de segunda vuelta: "{segunda_vuelta(DATA)}"')

Votación de pluralidad: "Sin piña"
Votación de borda: "Mejor tacos"
Votación de segunda vuelta: "Mejor tacos"


**Conclusión**

La votación de pluralidad es la que se utiliza usualmente, pero solo tiene en consideración el que sale más veces en primer lugar de todos los votantes. Es decir, como no considera el orden, podemos decir que la mayoría no quedará satisfecha con el resultado de la votación. 

Por otro lado, la votación con el método de borda considera el orden de preferencias de cada uno de los votantes. Es decir, pese a que no sea el candidato favorito de la mayoría de los votantes, el candidato que se elija va a ser el que tiene la mejor preferencia considerando a todos y cada uno de los votantes.

Por último, el de segunda vuelta también considera el orden de preferencia de los votantes, pero va eliminando al de menor preferencia para asegurarse de que el candidato elegido sea aquel que la mayoría prefiera.